## Installing Dependencies

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
  LoraConfig ,
  PeftConfig ,
  PeftModel ,
  get_peft_model ,
  prepare_model_for_kbit_training,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, AutoConfig

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## Downloading the original model

In [3]:
# MODEL_NAME = "tiiuae/falcon-7b-instruct"
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"
bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_use_double_quant = True,
  bnb_4bit_quant_type= "nf4",
  bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map = "auto",
    trust_remote_code = True,
    quantization_config = bnb_config
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

## Loading the PEFT Adapters

In [4]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
# print_trainable_parameters(model)

### Connecting the model and adapter

In [5]:
peft_model = "ayush-vatsal/caption_qlora_finetune"
config = PeftConfig.from_pretrained(peft_model)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict = True,
    quantization_config=bnb_config,
    device_map= "auto",
    trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token
model = PeftModel.from_pretrained(model, peft_model)

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

### Setting configs and prompting model

In [6]:
generation_config = model.generation_config
generation_config.max_new_tokens = 50
generation_config.temperature = 0.01
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id=tokenizer.eos_token_id
generation_config.eos_token_id=tokenizer.eos_token_id

In [9]:
def prompt(input_text="A man walking alone in the road"):
    return f"""
    Convert the given image description to social media worthy caption
    Description: {input_text}
    Caption:
    """.strip()

def get_social_media_caption(input_text="A man walking alone in the road"):
    device = "cuda:0"
    encoding = tokenizer(prompt(input_text), return_tensors = "pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids = encoding.input_ids,
            attention_mask = encoding.attention_mask,
            generation_config = generation_config
        )
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [10]:
out = get_social_media_caption("A woman standing in front of a temple")
query, captions = out.split('Caption: "')
captions, _ = captions.split('"')
print(captions)

In the presence of sacred architecture, a woman finds strength and peace. A temple is not just a structure, but a space for connection and reflection. 🌟✨
